In [60]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from cycler import cycler
from matplotlib.lines import Line2D 

from ipywidgets import (
    HBox, VBox, FloatText, FloatRangeSlider, IntSlider, FloatSlider, 
    Play, jslink, FloatLogSlider, interactive_output, HTML, HTMLMath, Layout, Checkbox, Label, Text
)

# global constants
eps0        = 8.854e-12     # [As/(Vm)] - permittivity of free space
q           = 1.602e-19     # [As]      - electronic charge
kb          = 1.38e-23      # [J/K]     - Boltzmann's constant

# global fixed parameters
T           = 300           # [K]       - temperature
UT          = kb*T/q        # [V]       - thermal voltage
epsr_si     = 11.7          # [ ]       - relative permittivity of Si
epsr_ox     = 3.73          # [ ]       - relative permittivity of SiO2
ni          = 1.07e10       # [cm^-3]    - intrinsic carrier concentration in Si (at 300 K)
Lch         = 0.5e-6        # [m]       - channel length
mu_n        = 0.1           # [m^2/(Vs)]- mobility of electrons in Si
Uth0        = 0.7           # [V]       - threshold voltage
UA          = 50            # [V]       - early voltage

# Bias point initialization
Ugs_sweep   = np.linspace(0,2.5,101)     # [V]   - gate-source voltage
Uds_sweep   = np.linspace(0,5,101)      # [V]   - drain-source voltage
Usb_sweep   = np.linspace(0,5,101)      # [V]   - source-substrate voltage

### Widget definitions
Uth0_wid = Label(value=r'Threshold voltage \(U_{\rm{th0}}\)'+' = '+str(Uth0)+' V') # threshold voltage - Uth0
Lch_wid = Label(value=r'Channel length \(L\)'+' = '+str(Lch*1e6)+' µm') # Channel length - L
mu_n_wid = Label(value=r'Electron mobility \(\mu_{\rm{n}}\)'+' = '+str(mu_n*1e4)+r' \(\rm{cm^{2}/(Vs)}\)') # Electron mobility 
UA_wid = Label(value=r'Early voltage \(U_{\rm{A}}\)'+' = '+str(UA)+' V') # Early voltage
ni_wid = Label(value=r'Intrinsic concentration  \(n_{\rm{i}}\)'+' = '+str(ni/1e10)+r' \(\rm{\times 10^{10} cm^{-3}}\)') # Intrinsic concentration
temp_wid = Label(value=r'Temperature \(T\)'+' = '+str(T)+' K') # Temperature

### Equations
IDeqn_wid = HTMLMath(
    value=r"Drain current: $$I_{\mathrm{D,lin}} = k'_{n}\left[(U_{GS}-U_{th})U_{DS} - (1+a_{th})\frac{U_{DS}^2}{2}\right]$$", # \text{for $U_{GS}<U_{th},U_{DS}\le U_{DS,sat}$}$$", # \\ I_{\mathrm{D}} = k'_{n}\left[\frac{1-k_{clm}}{2(1+a_{th})}(U_{GS}-U_{th})^2 + \frac{k_{clm}}{2}(U_{GS}-U_{th})U_{DS})\right] \text{for $U_{GS}<U_{th},U_{DS}>U_{DS,sat}$}\\ I_{\mathrm{D}} = 0 \text{ for $U_{GS}<U_{th}$}$$",
#     description='Drain current',
)


style = {'description_width': 'initial'}


### Variables and their slider definitions
# Width (W)
W_slider = FloatSlider(
    value=5,
    min=1,
    max=10,
    step=0.5,
    description=r'\(W/\rm{\mu m}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    # style=style
)

# Oxide thickness (d_ox)
dox_slider = FloatSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description=r'\(d_{\rm{ox}}/\rm{nm}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    # style=style
)

# Acceptor concentration (N_A)
NA_slider = FloatLogSlider(
    value=5e16,
    base=10,
    min=15, # max exponent of base
    max=17, # min exponent of base
    step=1e-1, # exponent step
    description=r'\(N_{\rm{A}}^{-}/\rm{cm^{-3}}\)',
    continuous_update=False,
    # style=style
)

# Terminal voltage and its slider definitions
# Gate-source voltage (U_GS) for Output characteristics
Ugs_slider = FloatRangeSlider(
    value=[1, 2.5],
    min=0.5,
    max=4.0,
    step=0.5,
    description='Parameter: $U_{\mathrm{GS}}$/V',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout={'width':'40%'},
    style=style
)
# Drain-source voltage (U_DS) for transfer characteristics
Uds_slider = FloatRangeSlider(
    value=[0.1, 3],
    min=0,
    max=5.0,
    step=0.1,
    description='Parameter: $U_{\mathrm{DS}}$/V',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout={'width':'40%'},
    style=style
)
# source-substrate voltage (U_SB) for plotting Uth vs Usb
Usb_slider = FloatSlider(
    value=0,
    min=np.min(Usb_sweep),
    max=np.max(Usb_sweep),
    step=0.5,
    description='$U_{\mathrm{SB}}$/V',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout={'width':'40%'}
    # style=style
)

# function definitions
def calc_Cox(d_ox):
    """Function to oxide capacitance C_ox
    Input
    ------
    d_ox    : float
        oxide thickness.
    Output
    ---------
    C_ox    : float
        oxide capacitance per unit area.
    """
    d_ox_m = d_ox
    C_ox = eps0*epsr_ox/d_ox_m
    return C_ox

def calc_kn(W,L,mu_n):
    """Function to calculate the drain current prefactor.
    Input
    ------
    W       : float 
        gate width.
    L       : float 
        channel length.
    mu_n    : float
        electron mobility.
    Output
    ---------
    km      : float
        drain current prefactor (per unit area).
    """
    C_ox = calc_Cox(dox_slider.value*1e-9)
    
    kn = W/L*mu_n*C_ox
    return kn

def calc_gamma():
    """Function to calculate gamma, i.e. body effect coefficient (see 2.5.19 in [1]).
    Input
    ------
    None
    Output
    ---------
    gamma   : float
        body effect coefficient.
    """
    C_ox = calc_Cox(dox_slider.value*1e-9)
    
    gamma = (1/C_ox)*np.sqrt(2*eps0*epsr_si*q*NA_slider.value*1e6)
    return gamma

def calc_phib():
    """Function to calculate phi_b, i.e. the upper limit of weak inversion (see 2.5.25 in [1]).
    Input
    ------
    None
    Output
    ---------
    phi_b   : float
        the upper limit of weak inversion.
    """
    phi_f = UT*np.log(NA_slider.value/ni)

    phi_b = 2*phi_f
    return phi_b

def calc_ath(Usb):
    """Function to calculate a_th, i.e. the slope of extrapolated threshold voltage Uth vs Usb (see 4.4.33b in [1]).
    Input
    ------
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    a_th    : float
        slope of extrapolated threshold voltage Uth vs Usb.
    """
    gamma   = calc_gamma()
    phi_b   = calc_phib()

    a_th = 0.5*gamma/np.sqrt(phi_b+Usb)
    return a_th

def calc_Uth(Uth0,Usb):
    """Function to calculate Uth, i.e. gate-source extrapolated threshold voltage (see 4.4.26b in [1]).
    Input
    ------
    Uth0     : float 
        threshold voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    Uth    : float
        gate-source extrapolated threshold voltage.
    """
    gamma   = calc_gamma()
    phi_b   = calc_phib()

    Uth = Uth0 + gamma*(np.sqrt(phi_b+Usb) - np.sqrt(phi_b))
    return Uth

def calc_ID(Ugs,Uds,Usb):
    """Function to calculate the drain current ID for single op point (see 4.4.30 in [1]).
    Input
    ------
    Ugs     : float 
        gate-source voltage.
    Uds     : float 
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID      : float
        Drain current.
    """

    kn      = calc_kn(W_slider.value*1e-6,Lch,mu_n)
    a_th    = calc_ath(Usb)
    Uth     = calc_Uth(Uth0,Usb)
    
    Uds_sat = (Ugs-Uth)/(1+a_th)
    k_clm   = Uds_sat/(UA+Uds_sat)

    if Ugs < Uth:
        ID = 0
    else:
        if Uds <= Uds_sat:
            ID = kn*((Ugs-Uth)*Uds - 0.5*(1+a_th)*Uds**2)
        else:
            ID = kn*((1-k_clm)/(2*(1+a_th))*(Ugs-Uth)**2 + 0.5*k_clm*(Ugs-Uth)*Uds)

    # print(ID)
    return ID

def calc_ID_gds(Ugs,Uds,Usb):
    """Function to calculate the output characteristics and output conductance
    Input
    ------
    Ugs     : list 
        gate-source voltage.
    Uds     : ndarray 
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID_out  : ndarray
        Drain current (output characteristics).
    gds     : ndarray
        Output conductance.
    """

    ID_out  = np.zeros(len(Uds))
    gds     = np.zeros(len(Uds))

    for nUds in np.arange(np.size(Uds)):
        ID_out[nUds] = calc_ID(Ugs,Uds[nUds],Usb)
    gds = np.gradient(ID_out)/np.gradient(Uds)

    return ID_out,gds

def calc_ID_gm(Ugs,Uds,Usb):
    """Function to calculate the transfer characteristics and transconductance
    Input
    ------
    Ugs     : ndarray  
        gate-source voltage.
    Uds     : list
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID_trans    : ndarray
        Drain current (transfer characteristics).
    gm          : ndarray
        transconductance.
    """

    ID_trans    = np.zeros((len(Uds),len(Ugs)))
    gm          = np.zeros((len(Uds),len(Ugs)))

    for nUds in np.arange(np.size(Uds)):
        for nUgs in np.arange(np.size(Ugs)):
            ID_trans[nUds,nUgs] = calc_ID(Ugs[nUgs],Uds[nUds],Usb)
        gm[nUds,:]  = np.gradient(ID_trans[nUds,:])/np.gradient(Ugs)
    
    return ID_trans,gm

def calc_gmb(Ugs,Uds,Usb):
    """Function to calculate the substrate transconductance
    Input
    ------
    Ugs     : ndarray  
        gate-source voltage.
    Uds     : list
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    gmb          : ndarray
        substrate transconductance.
    """
    
    ID_sub    = np.zeros((len(Uds),len(Usb)))
    gmb       = np.zeros((len(Uds),len(Usb)))

    for nUds in np.arange(np.size(Uds)):
        for nUsb in np.arange(np.size(Usb)):
            ID_sub[nUds,nUsb] = calc_ID(Ugs,Uds[nUds],Usb[nUsb])
        gmb[nUds,:]  = np.gradient(ID_sub[nUds,:])/np.gradient(Usb)
    
    return gmb

def plot_Uth_gmb(Ugs,Uds,Usb):
    
    # calculate current and gm
    gmb = calc_gmb(Ugs,Uds,Usb)
    
    ax1_ID_gm.cla()
    ax2_ID_gm.cla()
    
    # plot lines
    # ID
    ax1_ID_gm.plot(Ugs, ID[0]*1e3, label=str(Uds[0]))
    ax1_ID_gm.plot(Ugs, ID[1]*1e3, label=str(Uds[1]))
    # gm
    ax2_ID_gm.plot(Ugs, gm[0]*1e3, label=str(Uds[0]))
    ax2_ID_gm.plot(Ugs, gm[1]*1e3, label=str(Uds[1]))

    # set legend and label
    ax1_ID_gm.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
    ax1_ID_gm.set_xlabel('$U_{\mathrm{GS}}\mathrm{/V}\;→$')
    ax1_ID_gm.set_ylabel('$I_{\mathrm{D}}\mathrm{/mA}\;→$')
    
    ax2_ID_gm.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
    ax2_ID_gm.set_xlabel('$U_{\mathrm{GS}}\mathrm{/V}\;→$')
    ax2_ID_gm.set_ylabel('$g_{\mathrm{m}}\mathrm{/mS}\;→$')
    
    # set title
    ax1_ID_gm.set_title('$I_{\mathrm{D}}(U_{\mathrm{GS}})$')
    ax2_ID_gm.set_title('$g_{\mathrm{m}}(U_{\mathrm{GS}})$')
    
    # set limits
    ax1_ID_gm.set_xlim(0, np.max(Ugs))
    ax2_ID_gm.set_xlim(0, np.max(Ugs))
    ax1_ID_gm.set_ylim(ymin=0)
    ax2_ID_gm.set_ylim(ymin=0)
    
    fig_ID_gm.canvas.draw()
    fig_ID_gm.canvas.flush_events()

def plot_ID_gds(Ugs,Uds,Usb):
    
    UgsMin = np.min(Ugs)
    ax1_ID_gds.cla()
    ax2_ID_gds.cla()
    [ID,gds] = calc_ID_gds(UgsMin,Uds,Usb)
    ax1_ID_gds.plot(Uds, ID*1e3, label=str(UgsMin))
    ax2_ID_gds.plot(Uds, gds*1e3, label=str(UgsMin))
    for UgsVal in Ugs[1:]:
        [ID,gds] = calc_ID_gds(UgsVal,Uds,Usb)
        ax1_ID_gds.plot(Uds, ID*1e3, label=str(UgsVal))
        ax2_ID_gds.plot(Uds, gds*1e3, label=str(UgsVal))
    ax1_ID_gds.set_xlabel('$U_{\mathrm{DS}}\mathrm{/V}\;→$')
    ax1_ID_gds.set_ylabel('$I_{\mathrm{D}}\mathrm{/mA}\;→$')
    ax2_ID_gds.set_xlabel('$U_{\mathrm{DS}}\mathrm{/V}\;→$')
    ax2_ID_gds.set_ylabel('$g_{\mathrm{ds}}\mathrm{/mS}\;→$')
#     ax2_ID_gds.yaxis.tick_right()
#     ax2_ID_gds.yaxis.set_label_position("right")
    ax2_ID_gds.legend(loc='center left', bbox_to_anchor=(1.05, 0.5),title='$U_{\mathrm{GS}}\mathrm{/V}$')
    
    ax1_ID_gds.set_xlim(0, np.max(Uds))
    ax2_ID_gds.set_xlim(0, np.max(Uds))
    ax1_ID_gds.set_ylim(ymin=0)
    ax2_ID_gds.set_ylim(ymin=0)
    
    # set title
    ax1_ID_gds.set_title('$I_{\mathrm{D}}(U_{\mathrm{DS}})$')
    ax2_ID_gds.set_title('$g_{\mathrm{ds}}(U_{\mathrm{DS}})$')
    
    fig_ID_gds.canvas.draw()
    fig_ID_gds.canvas.flush_events()
    
def plot_ID_gm(Ugs,Uds,Usb):
    
    # calculate current and gm
    [ID,gm] = calc_ID_gm(Ugs,Uds,Usb)
    
    ax1_ID_gm.cla()
    ax2_ID_gm.cla()
    
    # plot lines
    # ID
    ax1_ID_gm.plot(Ugs, ID[0]*1e3, label=str(Uds[0]))
    ax1_ID_gm.plot(Ugs, ID[1]*1e3, label=str(Uds[1]))
    # gm
    ax2_ID_gm.plot(Ugs, gm[0]*1e3, label=str(Uds[0]))
    ax2_ID_gm.plot(Ugs, gm[1]*1e3, label=str(Uds[1]))

    # set legend and label
    ax1_ID_gm.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
    ax1_ID_gm.set_xlabel('$U_{\mathrm{GS}}\mathrm{/V}\;→$')
    ax1_ID_gm.set_ylabel('$I_{\mathrm{D}}\mathrm{/mA}\;→$')
    
    ax2_ID_gm.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
    ax2_ID_gm.set_xlabel('$U_{\mathrm{GS}}\mathrm{/V}\;→$')
    ax2_ID_gm.set_ylabel('$g_{\mathrm{m}}\mathrm{/mS}\;→$')
    
    # set title
    ax1_ID_gm.set_title('$I_{\mathrm{D}}(U_{\mathrm{GS}})$')
    ax2_ID_gm.set_title('$g_{\mathrm{m}}(U_{\mathrm{GS}})$')
    
    # set limits
    ax1_ID_gm.set_xlim(0, np.max(Ugs))
    ax2_ID_gm.set_xlim(0, np.max(Ugs))
    ax1_ID_gm.set_ylim(ymin=0)
    ax2_ID_gm.set_ylim(ymin=0)
    
    fig_ID_gm.canvas.draw()
    fig_ID_gm.canvas.flush_events()
    
def update_ID_gds(*args,**kwargs):
    [Ugs_op_min,Ugs_op_max] = Ugs_slider.value
    nOp = (Ugs_op_max-Ugs_op_min)/0.5 + 1
    Ugs_op = np.linspace(Ugs_op_min,Ugs_op_max,int(nOp))
    Usb_op = Usb_slider.value
    
    plot_ID_gds(Ugs_op,Uds_sweep,Usb_op)
    
def update_ID_gm(*args,**kwargs):
    """Function to update transfer characteristics and transconductance plots.
    """
    # get new op points from slider
    [Uds_op_min,Uds_op_max] = Uds_slider.value
    Uds_op = [Uds_op_min,Uds_op_max]
    Usb_op = Usb_slider.value
    
    plot_ID_gm(Ugs_sweep,Uds_op,Usb_op)
    
def update_outTransPlot(*args,**kwargs):
    """Function to update transfer characteristics and transconductance plots.
    """
    update_ID_gds()
    update_ID_gm()
    
plt.ioff()
plt.grid(True)
    
custom_cycler = (cycler(color=list('rgb')) *
       cycler(linestyle=['-', '--', '-.']))

plt.rc('lines', linewidth=1)
plt.rc('axes', prop_cycle=custom_cycler)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 1.5
plt.rcParams['grid.color'] = "#cccccc"

### Begin Uth vs Usb and gmb vs Usb fig
# fig_Uth_gmb = plt.figure(figsize=(8, 3.25))
# gs = fig_Uth_gmb.add_gridspec(1, 2, hspace=0.25, wspace=0.25, width_ratios=[0.9, 1], height_ratios=[1],bottom=0.13)
# (ax1_Uth_gmb, ax2_Uth_gmb) = gs.subplots(sharex=False, sharey=False)
# # fig.suptitle('Horizontally stacked subplots')
# fig_Uth_gmb.canvas.header_visible = False
# fig_Uth_gmb.canvas.layout.min_width = '400px'
# fig_Uth_gmb.canvas.toolbar_visible = True
# fig_Uth_gmb.canvas.capture_scroll = True

# box = ax2_Uth_gmb.get_position()
# ax2_Uth_gmb.set_position([box.x0, box.y0, box.width * 0.9, box.height])

# Uth = calc_Uth(Uth0,Usb_sweep)
# Usb_op  = 0

# plot_Uth_gmb(Ugs_op,Uds_sweep,Usb_op)

# line_Uth = Lines(
#     x             = Usb_sweep, 
#     y             = Uth, 
#     scales        = {'x': sc_x_Usb, 'y': sc_y_Uth},
#     display_legend= False,
#     labels        = ['Uth'],
# )
# index = np.argmin(np.abs(Usb_sweep-Usb_slider.value))
# line_op_usb = Lines(
#     x             = np.linspace(Usb_slider.value,Usb_slider.value,11), 
#     y             = np.linspace(1e-20, Uth[index],11), 
#     scales        = {'x': sc_x_Usb, 'y': sc_y_Uth},
#     display_legend= False,
#     colors        = ['MidnightBlue'],
# )

# fig_UthPlot     = Figure(marks=[line_Uth,line_op_usb], axes=[ax_x_Usb, ax_y_Uth], title='Uth(Usb)', layout=figLarge_layout)
# end Uth vs Usb fig

### Begin output characterisitics and output conductance fig
fig_ID_gds = plt.figure(figsize=(8, 3.25))
gs = fig_ID_gds.add_gridspec(1, 2, hspace=0.25, wspace=0.25, width_ratios=[0.9, 1], height_ratios=[1],bottom=0.13)
(ax1_ID_gds, ax2_ID_gds) = gs.subplots(sharex=False, sharey=False)
# fig_ID_gds.suptitle('$I_{\mathrm{D}}(U_{\mathrm{DS}})$'+' und '+'$g_{\mathrm{ds}}(U_{\mathrm{GS}})$')
fig_ID_gds.canvas.header_visible = False
fig_ID_gds.canvas.layout.min_width = '400px'
fig_ID_gds.canvas.toolbar_visible = True
fig_ID_gds.canvas.capture_scroll = True

box = ax2_ID_gds.get_position()
ax2_ID_gds.set_position([box.x0, box.y0, box.width * 0.9, box.height])

Ugs_op  = [1,1.5,2,2.5]
Usb_op  = 0

plot_ID_gds(Ugs_op,Uds_sweep,Usb_op)
# end output characterisitics and output conductance fig

### Begin transfer characterisitics and transconductance fig
fig_ID_gm = plt.figure(figsize=(8, 3.25))
gs = fig_ID_gm.add_gridspec(1, 2, hspace=0.25, wspace=0.25, width_ratios=[0.9, 1], height_ratios=[1],bottom=0.13)
(ax1_ID_gm, ax2_ID_gm) = gs.subplots(sharex=False, sharey=False)
# fig_ID_gm.suptitle('$I_{\mathrm{D}}(U_{\mathrm{GS}})$'+' und '+'$g_{\mathrm{m}}(U_{\mathrm{GS}})$')
fig_ID_gm.canvas.header_visible = False
fig_ID_gm.canvas.layout.min_width = '400px'
fig_ID_gm.canvas.toolbar_visible = True
fig_ID_gm.canvas.capture_scroll = True

box = ax2_ID_gm.get_position()
ax2_ID_gm.set_position([box.x0, box.y0, box.width * 0.9, box.height])

Uds_op = [0.1,3]
Usb_op = 0

plot_ID_gm(Ugs_sweep,Uds_op,Usb_op)
# end transfer characterisitics and transconductance fig

# observe sliders
# dox_slider.observe(update_all)
# NA_slider.observe(update_all)
# Usb_slider.observe(update_all)

W_slider.observe(update_outTransPlot)

Ugs_slider.observe(update_ID_gds)
Uds_slider.observe(update_ID_gm)


center_layout = Layout(
    display         = 'flex',
    flex_flow       = 'column',
    align_items     = 'center',
    width           = '100%',
    flex_grow       = 1,
    justify_content = 'space-around'
)

final_layout = VBox([HBox([fig_ID_gds.canvas,Ugs_slider],layout=center_layout),
                 HBox([fig_ID_gm.canvas,Uds_slider,W_slider],layout=center_layout),
                ])
final_layout

